In [1321]:
## Import necessary modules
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import AutoDateFormatter, AutoDateLocator, WeekdayLocator, MonthLocator, DateFormatter
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
from matplotlib.ticker import AutoMinorLocator
import numpy as np
import datetime, calendar
%matplotlib gtk

In [1322]:
## Import data frome pickle generated from hotline_daily.py
data = pd.read_pickle('pickle_hotline_0904.pkl')

In [1323]:
## Have a look at the data structure
data.head(3)

,month,year,0,1,2,3,4,5,6,7,...,21,22,23,24,summa,xlday,day,date,week,weekday
tix,11,2016,00:00-00:30,00:30-01:30,01:30-02:30,02:30-03:30,03:30-04:30,04:30-05:30,05:30-06:30,06:30-07:30,...,20:30-21:30,21:30-22:30,22:30-23:30,23:30-00:00,NaN,42688.0,14,2016-11-14,46,Mon
angekommen,11,2016,0,0,0,0,0,0,1,5,...,0,1,0,0,321,42688.0,14,2016-11-14,46,Mon
verbunden,11,2016,0,0,0,0,0,0,0,4,...,0,0,0,0,221,42688.0,14,2016-11-14,46,Mon


In [1324]:
##### HERE SOME FILTERING CAN TAKE PLACE
heute=datetime.date.today().strftime('%d.%m.%Y')

def weekfilt(year,week):
    df = data.ix[((data['year']==year) & (data['week']==week))]
    dataset_title='KW'+str(week)+' '+str(year)
    
    calls_d=df[['date','summa','weekday','day']].loc['angekommen'].set_index("date")
    verbu_d=df[['date','summa','weekday','day']].loc['verbunden'].set_index("date")
    verlo_d=df[['date','summa','weekday','day']].loc['verloren'].set_index("date")
    
    dayf = pd.DataFrame(index=calls_d.index)
    dayf[['angekommen','weekday','day']]=calls_d[['summa','weekday','day']]
    dayf['verbunden']=verbu_d['summa']
    dayf['verloren']=verlo_d['summa']
    dayf=dayf[['weekday','day','angekommen','verbunden','verloren']]
    
    return dayf,dataset_title
    
#data = data.ix[((data['year']==2017) & (data['week']==1))]
#dataset_title='KW01 2017'

#data = data.ix[((data['year']==2017) & (data['month']==1))]
#dataset_title='Nov 2016 - Apr 2017'

#data = data.ix[((data['year']==2017) & (data['month']==2))]
#dataset_title='Feb 2017'

data = data.ix[(data['year']==2017)]
dataset_title='2017 bis '+str(heute)

# without any filtering:
#dataset_title='Nov 2016 - now'

In [1325]:
## separate the sums of calls, verbundene and lost into different dataframes; this will avoid tuple indexing
calls_d=data[['date','summa','weekday','day']].loc['angekommen'].set_index("date")
verbu_d=data[['date','summa','weekday','day']].loc['verbunden'].set_index("date")
verlo_d=data[['date','summa','weekday','day']].loc['verloren'].set_index("date")

## have a look at one of the new dataframes
print calls_d.index
calls_d.head(2)

Index([2017-01-01, 2017-01-02, 2017-01-03, 2017-01-04, 2017-01-05, 2017-01-06,
       2017-01-07, 2017-01-08, 2017-01-09, 2017-01-10,
       ...
       2017-04-07, 2017-04-08, 2017-04-09, 2017-04-10, 2017-04-11, 2017-04-12,
       2017-04-13, 2017-04-14, 2017-04-15, 2017-04-16],
      dtype='object', name=u'date', length=106)


,summa,weekday,day
date,,,
2017-01-01,4,Sun,1
2017-01-02,103,Mon,2


In [1326]:
## merge dataframes again into one, containing the sums and indexed by date.
## this seems kind of redundant, given the frames above. guess it should be refactored; but works for now

dayf = pd.DataFrame(index=calls_d.index)
dayf[['angekommen','weekday','day']]=calls_d[['summa','weekday','day']]
dayf['verbunden']=verbu_d['summa']
dayf['verloren']=verlo_d['summa']
dayf=dayf[['weekday','day','angekommen','verbunden','verloren']]

## take a glance at the new dataframe
dayf.head(2)

,weekday,day,angekommen,verbunden,verloren
date,,,,,
2017-01-01,Sun,1,4,3,1
2017-01-02,Mon,2,103,87,16


In [1327]:
## group by weekday for an analysis of the average week
def av_week(d_frame):
    avweek = d_frame[['weekday','angekommen','verbunden','verloren']].groupby(['weekday']).sum()
    day_counter=dayf['weekday'].value_counts().reindex(list(calendar.day_abbr))
    avweek = avweek.reindex(list(calendar.day_abbr)).reset_index()
    avweek['daycount']=day_counter.values
    avweek['av_angekommen']=np.round(avweek['angekommen']/avweek['daycount'],decimals=2)
    return avweek

In [1328]:
awk=av_week(dayf)
awk

,weekday,angekommen,verbunden,verloren,daycount,av_angekommen
0,Mon,1202.0,1033.0,169.0,15,80.13
1,Tue,1140.0,1032.0,108.0,15,76.00
2,Wed,1035.0,918.0,117.0,15,69.00
3,Thu,1208.0,1088.0,120.0,15,80.53
4,Fri,1036.0,950.0,86.0,15,69.07
5,Sat,377.0,324.0,53.0,15,25.13
6,Sun,56.0,43.0,13.0,16,3.50


In [1329]:
## this will generate the ticks on the x-Axis of the plot
## I don't fully understand this, though. more reading is due
#xtick_locator = WeekdayLocator(byweekday=MO)
#xtick_formatter = DateFormatter('%W/%y')
#xtick_mon_loc = MonthLocator()
#xtick_mon_for = DateFormatter('%m/%y')
#print(xtick_locator)
#print(xtick_formatter)



In [1330]:
## Create the plot from the summary dataframe
## X-Axis should be dates, Y-Axis should be number of calls
def plot_daily_abs(d_frame,title):
    xtick_locator = WeekdayLocator(byweekday=MO)
    xtick_formatter = DateFormatter('%W/%y')
    xtick_mon_loc = MonthLocator()
    xtick_mon_for = DateFormatter('%b/%y')
    
    dayf = d_frame # just for the sake of not having to rewrite it all
    
    fig = plt.figure(figsize=(18,6))
    fig.add_subplot(111)
    plt.ylabel('Calls')
    plt.xlabel('Kalenderwoche')
    plt.margins(0.01)
    plt.title(title)

    # "Initialize" Axes to configure them later
    ax = plt.axes()
    ax.set_axisbelow(True)

    # Y-Axis: major ticks every 50, minor ticks every 10                                    
    minor_ticks = np.arange(0, dayf.angekommen.max(), 10)
    major_ticks = np.arange(0, dayf.angekommen.max(), 50)
    ax.yaxis.grid(True)
    ax.set_yticks(minor_ticks, minor=True)
    ax.yaxis.grid(True, which='minor', color='#E8E6BF', linestyle='--', alpha=0.5, zorder=0)
    ax.yaxis.grid(True, which='major', color='#6DC3EC', linestyle='-', zorder=3)

    # X-Axis: locator set the ticks. No understanding of this, just copy-and-paste...
    ax.xaxis.set_tick_params(which='major', pad=1, length=20)
    ax.xaxis.set_major_locator(xtick_mon_loc)
    ax.xaxis.set_major_formatter(xtick_mon_for)
    ax.xaxis.set_minor_locator(xtick_locator)
    ax.xaxis.set_minor_formatter(xtick_formatter)

    # Bars for each sum and day
    plt.bar(dayf.index,dayf.angekommen,width=0.9, alpha=1, color='#8F8F8F', label='angekommen', align='center')
    plt.bar(dayf.index,dayf.verbunden,width=0.2, alpha=1, color='#00B9FF', label='verbunden', align='center')
    plt.bar(dayf.index,dayf.verloren,width=0.2, alpha=1, color='#FF0056', label='verloren', align='center')

    # Markers for Mondays and Months
    plt.plot(dayf[dayf.weekday=='Mon'].index, dayf[dayf.weekday=='Mon'].day, c='#00FFEB', marker='d', markersize='4', linestyle='none', label='Montage')
    plt.plot(dayf[dayf.day==1].index, dayf[dayf.day==1].day, c='#FFF800', marker='^', markersize='4', linestyle='none', label='Monatsbeginn')

    # Legend and Layout formatting after the plotting has taken place
    plt.legend()
    plt.tight_layout()


In [1331]:
## Create the plot from the summary dataframe
## X-Axis should be dates, Y-Axis should be number of calls
def plot_weekly_abs(d_frame,title):
    xtick_locator = WeekdayLocator(byweekday=[MO,TU,WE,TH,FR,SA,SU])
    xtick_formatter = DateFormatter('%a,%d.%m.%y')
    dayf = d_frame # just for the sake of not having to rewrite it all
    
    fwidth=8
    fheight=6
    fig = plt.figure(figsize=(fwidth,fheight))
    
    ax=fig.add_subplot(111)
    ax.set_ylabel('Calls')
    ax.set_xlabel('Kalenderwoche '+title)
    ax.margins(0.01)
    ax.tick_params(axis='both', labelsize=fwidth)
    ax.set_axisbelow(True)

    # Y-Axis: major ticks every 50, minor ticks every 10                                    
    ax.yaxis.grid(True)
    ax.yaxis.set_minor_locator(AutoMinorLocator(10))
    ax.yaxis.grid(True, which='minor', color='#E8E6BF', linestyle='--', alpha=0.5, zorder=0)
    ax.yaxis.grid(True, which='major', color='#6DC3EC', linestyle='-', zorder=3)

    # X-Axis: locator set the ticks. No understanding of this, just copy-and-paste...
    ax.xaxis.set_major_locator(xtick_locator)
    ax.xaxis.set_major_formatter(xtick_formatter)
    
    # Another x-Axis for average number. 'bar' is needed to adjust spacing
    ax2 = ax.twiny()
    ax2.margins(0.01)
    ax2.set_xlabel('angekommene Calls')
    ax2.xaxis.set_major_locator(xtick_locator)
    top_labels=dayf.angekommen.values
    ax2.set_xticklabels(top_labels)
    ax2.bar(dayf.index,np.linspace(0,0,num=7),width=0.6, align='center')

    # Bars for each sum and day
    ax.bar(dayf.index,dayf.angekommen,width=0.6, alpha=1, color='#8F8F8F', label='angekommen', align='center')
    ax.bar(dayf.index,dayf.verbunden,width=0.4, alpha=1, color='#00B9FF', label='verbunden', align='center')
    ax.bar(dayf.index,dayf.verloren,width=0.2, alpha=1, color='#FF0056', label='verloren', align='center')

    # Legend and Layout formatting after the plotting has taken place
    ax.legend(prop={'size':fheight*2})
    fig.tight_layout()


In [1332]:
## Create the plot from the summary dataframe
## X-Axis should be dates, Y-Axis should be number of calls
def plot_monthly_abs(d_frame,title):
    xtick_locator = WeekdayLocator(byweekday=[MO,TU,WE,TH,FR,SA,SU])
    xtick_formatter = DateFormatter('%d.%m.')
    dayf = d_frame # just for the sake of not having to rewrite it all
    
    fwidth=10
    fheight=5
    fig = plt.figure(figsize=(fwidth,fheight))
    
    ax=fig.add_subplot(111)
    ax.set_ylabel('Calls')
    ax.set_xlabel('Tage '+title)
    ax.margins(0.01)
    ax.tick_params(axis='both', labelsize=fwidth*0.6)
    ax.set_axisbelow(True)

    # Y-Axis: major ticks every 50, minor ticks every 10                                    
    ax.yaxis.grid(True)
    ax.yaxis.set_minor_locator(AutoMinorLocator(10))
    ax.yaxis.grid(True, which='minor', color='#E8E6BF', linestyle='--', alpha=0.5, zorder=0)
    ax.yaxis.grid(True, which='major', color='#6DC3EC', linestyle='-', zorder=3)

    # X-Axis: locator set the ticks. No understanding of this, just copy-and-paste...
    ax.xaxis.set_major_locator(xtick_locator)
    ax.xaxis.set_major_formatter(xtick_formatter)
    
    # Another x-Axis for average number. 'bar' is needed to adjust spacing
    ax2 = ax.twiny()
    ax2.margins(0.01)
    ax2.set_xlabel('angekommene Calls')
    ax2.tick_params(labelsize=fwidth*0.6)
    ax2.xaxis.set_major_locator(xtick_locator)
    top_labels=dayf.angekommen.values
    ax2.set_xticklabels(top_labels)
    ax2.bar(dayf.index,np.linspace(0,0,num=len(dayf.index)),width=0.6, align='center')

    # Bars for each sum and day
    ax.bar(dayf.index,dayf.angekommen,width=0.6, alpha=1, color='#8F8F8F', label='angekommen', align='center')
    ax.bar(dayf.index,dayf.verbunden,width=0.4, alpha=1, color='#00B9FF', label='verbunden', align='center')
    ax.bar(dayf.index,dayf.verloren,width=0.2, alpha=1, color='#FF0056', label='verloren', align='center')
    
    # Markers for Mondays and Months
    ax.plot(dayf[dayf.weekday=='Mon'].index, dayf[dayf.weekday=='Mon'].day, c='#00FFEB', marker='d', markersize=fheight, linestyle='none', label='Montage')
    #plt.plot(dayf[dayf.day==1].index, dayf[dayf.day==1].day, c='#FFF800', marker='^', markersize='4', linestyle='none', label='Monatsbeginn')

    # Legend and Layout formatting after the plotting has taken place
    ax.legend(prop={'size':fheight*2})
    fig.tight_layout()

In [1333]:
## Create the plot from the summary dataframe
## X-Axis should be dates, Y-Axis should be number of calls
def plot_average_week(d_frame,title):
    
    dayf = d_frame # just for the sake of not having to rewrite it all
    fwidth=8
    fheight=6
    fig = plt.figure(figsize=(fwidth,fheight))
    ax=fig.add_subplot(111)
    ax.set_ylabel('Gesamtanzahl Calls')
    
    #ax.margins(0.01)
    

    # "Initialize" Axes to configure them later
    # ax = plt.axes()
    

    # Y-Axis: set minor ticks to every 10 of major                                 
    ax.yaxis.set_minor_locator(AutoMinorLocator(10))
    ax.yaxis.grid(True)
    ax.yaxis.grid(True, which='minor', color='#E8E6BF', linestyle='--', alpha=0.5, zorder=0)
    ax.set_axisbelow(True)
    #ax.set_yticks(minor_ticks, minor=True)
    #ax.set_yticks(major_ticks)
    
    #ax.yaxis.grid(True, which='major', color='#6DC3EC', linestyle='-', zorder=3)

    # X-Axis: locator set the ticks. No understanding of this, just copy-and-paste...
    #ax.xaxis.set_major_locator(xtick_locator)
    #ax.xaxis.set_major_formatter(xtick_formatter)
    ax.set_xlabel('Wochentag')
    ax.set_xticks(np.arange(0,7,1))
    ax.set_xticklabels(dayf.weekday)
    
    
    # Another x-Axis for average number
    ax2 = ax.twiny()
    ax2.set_xlabel('durchschnitl. Calls / Wochentag '+title)
    ax2.set_xticks(np.arange(0,7,1))
    top_labels=dayf.av_angekommen.values
    ax2.set_xticklabels(top_labels)
    #ax2.set_xticklabels(['here','needs','to','be','the','average','value'])
    
    # Bars for each sum and day
    ax.bar(dayf.index,dayf.angekommen,width=0.9, alpha=1, color='#8F8F8F', label='angekommen', align='center')
    ax.bar(dayf.index,dayf.verbunden,width=0.2, alpha=1, color='#00B9FF', label='verbunden', align='center')
    ax.bar(dayf.index,dayf.verloren,width=0.2, alpha=1, color='#FF0056', label='verloren', align='center')

    ax2.bar(dayf.index,np.arange(0,7),width=0.9, alpha=1, color='#8F8F8F', label='angekommen', align='center')
    # Markers for Mondays and Months
    #plt.plot(dayf[dayf.weekday=='Mon'].index, dayf[dayf.weekday=='Mon'].day, c='#00FFEB', marker='d', markersize='4', linestyle='none', label='Montage')
    #plt.plot(dayf[dayf.day==1].index, dayf[dayf.day==1].day, c='#FFF800', marker='^', markersize='4', linestyle='none', label='Monatsbeginn')

    # Legend and Layout formatting after the plotting has taken place
    ax.legend(prop={'size':fheight*2})
    fig.tight_layout()


In [1334]:
## This is for an average week. Data grouped by weekdays. 7 bars from Mon to Sun

#average_week=av_week(dayf)
#print average_week.av_angekommen
#plot_average_week(average_week,dataset_title)


In [1335]:
## This is for single weeks. It expects a week as an input. Produces Output only for single weeks

#plot_weekly_abs(dayf,dataset_title)

In [1336]:
## This is for single months. Gives absolute numbers for each date.
## Reasonsable output for single month, but works with larger ranges as well

#plot_monthly_abs(dayf,dataset_title)

In [1337]:
## Shows daily absolute numbers... like monthly I guess. Wonder what my intention was here. 
## Ticks are calendar weeks and months. Works for longer time intervals like quartal, halfyear

#plot_daily_abs(dayf,dataset_title)

In [1338]:
## go through a range of weeks

#for week in range(1,15):
    #dayf,titel=weekfilt(2017,week)
    #plot_weekly_abs(dayf,titel)